### Grand et petit axe 3D (01 -> 08 and 18)

In [1]:
import SimpleITK as sitk
import csv
from pathlib import Path

data_path = Path.cwd().parent.parent / "data"

mask_int_path = data_path / "08_mask_int_nii_gz_dir"
if not mask_int_path.exists():
    mask_int_path.mkdir(parents = True, exist_ok = True)
    
major_minor_axis_dir = data_path / "18_3D_major_minor_axis_csv_dir"
if not major_minor_axis_dir.exists():
    major_minor_axis_dir.mkdir(parents = True, exist_ok = True)

In [2]:
"""
01 -> 08
Convert mask to int and store output into 08_mask_int_results
This script is necessary for major and minor axes because MASKs in 01_MSLesSeg_Dataset are 32-bit float which are not
supported in 3D 
"""

mslesseg_path = data_path / "01_MSLesSeg_Dataset"

# Convert train data mask into int
train_irm_paths = list(mslesseg_path.glob("train/*/*"))

for path in train_irm_paths:
    mask_path = list(path.glob("*_MASK.nii.gz"))
    
    if mask_path:
        mask = sitk.ReadImage(mask_path[0])
        mask_int = sitk.Cast(mask, sitk.sitkInt16)
        
        title, format = mask_path[0].name.split(".", 1)
        rename = title + "_INT." + format
        output_path = mask_int_path / rename
        sitk.WriteImage(mask_int, output_path)

# Convert test data mask into int
test_irm_paths = list(mslesseg_path.glob("test/*"))

for path in test_irm_paths:
    mask_path = list(path.glob("*_MASK.nii.gz"))
    
    if mask_path:
        mask = sitk.ReadImage(mask_path[0])
        mask_int = sitk.Cast(mask, sitk.sitkInt16)
        
        title, format = mask_path[0].name.split(".", 1)
        rename = title + "_INT." + format
        output_path = mask_int_path / rename
        sitk.WriteImage(mask_int, output_path)

In [3]:
"""
08 -> 18
Generate major and minor axis data csv and store into 18_3D_major_minor_axis_csv_dir
"""

for path in mask_int_path.rglob("*_MASK_INT.nii.gz"):

    arg = path.stem.split("_")

    if len(arg) == 4:
        patient_id, timepoint = arg[0], arg[1]
        out_csv = major_minor_axis_dir / f"{patient_id}_{timepoint}_major_minor_axis.csv"

    elif len(arg) == 3:
        patient_id, timepoint = arg[0], "T1"
        out_csv = major_minor_axis_dir / f"{patient_id}_major_minor_axis.csv"

    else:
        continue


    # ============================
    # Lecture image
    # ============================
    mask_int = sitk.ReadImage(path)

    cc = sitk.ConnectedComponent(mask_int > 0)


    # ============================
    # Shape statistics
    # ============================
    shape_stats = sitk.LabelShapeStatisticsImageFilter()
    shape_stats.Execute(cc)


    # ============================
    # Collecte des lésions
    # ============================
    lesions = []

    for label in shape_stats.GetLabels():

        volume = shape_stats.GetPhysicalSize(label)

        ellipsoid = shape_stats.GetEquivalentEllipsoidDiameter(label)

        minor_axis = min(ellipsoid)
        major_axis = max(ellipsoid)

        lesions.append({
            "old_label": label,
            "volume": volume,
            "major": major_axis,
            "minor": minor_axis
        })


    # ============================
    # Tri par volume décroissant
    # ============================
    lesions_sorted = sorted(
        lesions,
        key=lambda x: x["volume"],
        reverse=True
    )


    # ============================
    # Écriture CSV
    # ============================
    with open(out_csv, "w", newline="", encoding="utf-8") as f:

        writer = csv.writer(f)

        # En-tête
        writer.writerow([
            "patient_id",
            "timepoint",
            "lesion_id",
            "volume_mm3",
            "major_axis_mm",
            "minor_axis_mm",
        ])


        # Nouveau label
        for new_id, lesion in enumerate(lesions_sorted, start=1):

            writer.writerow([
                patient_id,
                timepoint,
                new_id,
                f"{lesion['volume']:.1f}",
                f"{lesion['major']:.1f}",
                f"{lesion['minor']:.1f}",
            ])


    print(f"[OK] CSV généré : {out_csv.name}")


[OK] CSV généré : P10_T1_major_minor_axis.csv
[OK] CSV généré : P10_T2_major_minor_axis.csv
[OK] CSV généré : P11_T1_major_minor_axis.csv
[OK] CSV généré : P11_T2_major_minor_axis.csv
[OK] CSV généré : P12_T1_major_minor_axis.csv
[OK] CSV généré : P12_T2_major_minor_axis.csv
[OK] CSV généré : P12_T3_major_minor_axis.csv
[OK] CSV généré : P12_T4_major_minor_axis.csv
[OK] CSV généré : P13_T1_major_minor_axis.csv
[OK] CSV généré : P13_T2_major_minor_axis.csv
[OK] CSV généré : P14_T1_major_minor_axis.csv
[OK] CSV généré : P14_T2_major_minor_axis.csv
[OK] CSV généré : P14_T3_major_minor_axis.csv
[OK] CSV généré : P14_T4_major_minor_axis.csv
[OK] CSV généré : P15_T1_major_minor_axis.csv
[OK] CSV généré : P16_T1_major_minor_axis.csv
[OK] CSV généré : P17_T1_major_minor_axis.csv
[OK] CSV généré : P18_T1_major_minor_axis.csv
[OK] CSV généré : P19_T1_major_minor_axis.csv
[OK] CSV généré : P19_T2_major_minor_axis.csv
[OK] CSV généré : P19_T3_major_minor_axis.csv
[OK] CSV généré : P19_T4_major_min